In [ ]:
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten,InputLayer,AveragePooling2D, BatchNormalization

from keras import optimizers
from keras.models import load_model

In [ ]:
input_shape = (88, 20, 1)

In [ ]:
model = Sequential()

In [ ]:
model.add(InputLayer(input_shape = input_shape))

model.add(Conv2D(32, (6,6), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (4,4), activation="relu"))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))


model.add(Dense(5, activation='softmax'))

In [ ]:
model.load_weights("model_weights_output_92.h5")

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
import DataProvider
prov = DataProvider.DataProvider(False, maxLength= input_shape[0], n_mfcc= input_shape[1])

In [ ]:
X2 = list()
y2 = list()
actualVal = list()
sequence_of_test_data = list()

In [ ]:
while prov.hasNext():
    X2t, y2t = prov.next()
    file_name, file_number = y2t
    
    actualVal.append(file_number)
    sequence_of_test_data.append(file_name)
    
    k = np.zeros(5)
    k[file_number - 1] = 1
    
    X2.append(X2t)
    y2.append(k)

In [ ]:
X2 = np.nan_to_num(np.array(X2))
y2 = np.matrix(y2)

X2.shape, y2.shape

In [ ]:
X_test = X2.reshape(X2.shape[0], X2.shape[1], X2.shape[2], 1)
X_test.shape

In [ ]:
loss_and_metrics = model.evaluate(X_test, y2, batch_size=128)

In [ ]:
pred = model.predict(X_test, batch_size=128)

In [ ]:
pr = np.argmax(pred, axis=1)+1
pr , np.array(actualVal)

In [ ]:
correct = [1 if a == b else 0 for (a, b) in zip(pr, actualVal)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ("Accuracy: " + str(accuracy * 100) + "%")

In [ ]:
for i in pred:
    count = sum(i)
    for j in range(len(i)):
        i[j] = i[j] / count

In [ ]:
import copy
countGrade = copy.deepcopy(actualVal)

In [ ]:
for i in range(len(countGrade)):
    ans = actualVal[i]
    countGrade[i] = pred[i][ans-1]

In [ ]:
result = sum(countGrade) / len(countGrade)
result

In [ ]:
all_test_data_list = list()
for test_data_index in range(len(pred)):
    
    test_data_prediction_list = list()
    test_data_prediction_list.append(sequence_of_test_data[test_data_index])
    
    prediction = pred[test_data_index]
    prediction = prediction/prediction.sum()
    
    for prediciton_percent in prediction:
        test_data_prediction_list.append(prediciton_percent)
        
    all_test_data_list.append(test_data_prediction_list)

In [ ]:
import pandas as pd

df = pd.DataFrame(data=all_test_data_list, columns=['name','1','2','3','4','5'])

In [ ]:
df.to_excel("5.xlsx")